In [1]:
import datetime

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext
print(SparkContext.version)

In [4]:
sc = SparkContext()

Exception: Java gateway process exited before sending its port number

In [ ]:
sc

In [ ]:
import pickle
from urllib.request import urlopen
import os
import pandas as pd
import numpy as np
import gensim
import pickle
import matplotlib.pyplot as plt

In [ ]:
os.chdir("D://Datasets")

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Galeforse/DST-Assessment-05/main/Data/NCSC%20Reports.csv')

In [ ]:
df.head()

In [ ]:
print(np.shape(df))

In [ ]:
df.iloc[4,2]

In [ ]:
import nltk
from nltk.corpus import stopwords

In [ ]:
l1 = stopwords.words('english')
l2 = ['have', 'with', 'cannot', 'all', 'more', 'less', 'fewer', 'if', 'from', 'new', 'old', 'be', 'This']
l1.append(l2)
lst1 = [' {} '.format(x) for x in l1]

len(lst1)

In [ ]:
df['Article'].replace(lst1, " ", regex=True, inplace=True)
df['Article'].replace(r'[^\w\s]', " ", regex=True, inplace=True)
df['Article'].replace('[0-9999999999]', " ", regex=True, inplace=True)

In [ ]:
df.iloc[4,2]

In [ ]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [ ]:
# nltk.download('wordnet')
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [ ]:
df_lem = df['Article'].apply(lemmatize_text)

In [ ]:
df_lem

In [ ]:
df_lem_list = df_lem.values.tolist()

In [ ]:
lister = [[] for x in range(len(df_lem))]
for i in range(len(df_lem_list)):
    l = df_lem_list[i]
    for token in l:
        if (len(token)>4 or token.isupper()):
            lister[i].append(token)

lister[0:1]

In [ ]:
dictionary = gensim.corpora.Dictionary(lister)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in lister]
tfidf=gensim.models.TfidfModel(bow_corpus)
tfidf_corp = tfidf[bow_corpus]

In [ ]:
bow_corpus[0:1]

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(tfidf_corp, num_topics=6, id2word=dictionary, passes=2, workers=4)

In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [ ]:
lda_display = pyLDAvis.gensim.prepare(lda_model_tfidf, tfidf_corp, 
                                          dictionary, mds='mmds')
pyLDAvis.display(lda_display, template_type='notebook')

In [ ]:
from gensim.models.coherencemodel import CoherenceModel
def getCoherence(m,t,d):
    coherence_model_lda = CoherenceModel(model=m,texts=t, dictionary=d, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    return(coherence_lda) # Need high coherence

In [ ]:
getCoherence(m=lda_model_tfidf, t=lister, d =dictionary)

In [ ]:
perplex = lda_model_tfidf.log_perplexity(tfidf_corp)

In [ ]:
perplex

In [ ]:
begin_time = datetime.datetime.now()
gCH = [0] * 50
for i in range(1,50):
    locals()['lda_model_tfidf_{}'.format(i)] = gensim.models.LdaMulticore(tfidf_corp, num_topics=i, id2word=dictionary, passes=2, workers=6)
    gCH[i]=getCoherence(m=locals()['lda_model_tfidf_{}'.format(i)], t=lister, d =dictionary)
end_time = datetime.datetime.now()
print(end_time - begin_time)

In [ ]:
begin_time = datetime.datetime.now()
perplexity = [0] * 50
for i in range(1,50):
    perplexity[i]=locals()['lda_model_tfidf_{}'.format(i)].log_perplexity(tfidf_corp)
end_time = datetime.datetime.now()
print(end_time - begin_time)

In [ ]:
plt.plot(gCH)
plt.show()
plt.plot(perplexity)
plt.show()

In [ ]:
print("The maximum coherence achieved is", max(gCH), "and its corresponding number of topics is", np.argmax(gCH))
print("The minimum perplexity achieved is", min(perplexity), "and its corresponding number of topics is", np.argmin(perplexity))

In [ ]:
lda_display = pyLDAvis.gensim.prepare(lda_model_tfidf_18, tfidf_corp, 
                                          dictionary, mds='mmds')
pyLDAvis.display(lda_display, template_type='notebook')